In [1]:
import pandas as pd
from scipy.sparse import csr_matrix
import gc  # coleta de lixo para liberar memória

c:\Users\almei\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
# === 1. Carregamento dos arquivos ===
# ratings = pd.read_csv("bases/USER_RATINGS.csv")
games = pd.read_csv("bases/GAMES.csv")
mechanics = pd.read_csv("bases/MECHANICS.csv")

In [3]:
# ============================================================
# 1. Leitura otimizada da base GAMES (usada para filtro por ano)
# ============================================================

# Filtrar apenas jogos lançados a partir de 1990
modern_games = games[games["YearPublished"] >= 1990]["BGGId"]
modern_games_set = set(modern_games)  # conversão para lookup rápido
print(f"Número de jogos modernos (pós 1990): {len(modern_games_set):,}")

Número de jogos modernos (pós 1990): 19,111


In [4]:
# ============================================================
# 2. Leitura em CHUNKS da base USER_RATINGS (grande, 390 MB)
# ============================================================

chunksize = 1_000_000  # lê 1 milhão de linhas por vez
filtered_chunks = []

dtypes = {
    "BGGId": "int32",
    "Rating": "float32",
    "Username": "category"
}

for chunk in pd.read_csv(
    "bases/USER_RATINGS.csv",
    usecols=["Username", "BGGId", "Rating"],
    dtype=dtypes,
    chunksize=chunksize
):
    # Filtrar jogos modernos dentro de cada chunk
    chunk = chunk[chunk["BGGId"].isin(modern_games_set)]
    filtered_chunks.append(chunk)

# Concatenar tudo em um único DataFrame filtrado
ratings = pd.concat(filtered_chunks, ignore_index=True)
del filtered_chunks
gc.collect()

print(f"Após o filtro de ano: {len(ratings):,} avaliações")

Após o filtro de ano: 17,538,477 avaliações


In [5]:
# ============================================================
# 3. Filtragem adicional por número mínimo de avaliações
# ============================================================

# Filtrar usuários com pelo menos 10 avaliações
user_counts = ratings["Username"].value_counts()
active_users = user_counts[user_counts >= 10].index
ratings = ratings[ratings["Username"].isin(active_users)]

# Filtrar jogos com pelo menos 20 avaliações
game_counts = ratings["BGGId"].value_counts()
popular_games = game_counts[game_counts >= 20].index
ratings = ratings[ratings["BGGId"].isin(popular_games)]

print(f"Base final: {len(ratings):,} avaliações, "
      f"{ratings['Username'].nunique():,} usuários e "
      f"{ratings['BGGId'].nunique():,} jogos.")

Base final: 16,958,788 avaliações, 217,775 usuários e 19,070 jogos.


In [6]:
import psutil, os

processo = psutil.Process(os.getpid())
print(f"Memória usada: {processo.memory_info().rss / 1024 ** 2:.2f} MB")


Memória usada: 740.36 MB


### OLD

In [7]:
# === 2. Filtragem temporal ===
# Mantém apenas jogos lançados a partir da década de 1990
# games_modern = games[games["YearPublished"] >= 1990]

In [8]:
# === 3. Filtragem dos ratings ===
# Mantém apenas avaliações de jogos modernos
#ratings = ratings[ratings["BGGId"].isin(games_modern["BGGId"])]

In [9]:
# # === 4. Filtragem de usuários e jogos com baixa atividade ===
# # Define limites mínimos (ajustáveis conforme o tamanho final desejado)
# min_user_ratings = 20     # usuários com menos de 20 avaliações são removidos
# min_game_ratings = 50     # jogos com menos de 50 avaliações são removidos

# user_counts = ratings["Username"].value_counts()
# game_counts = ratings["BGGId"].value_counts()

# ratings = ratings[
#     ratings["Username"].isin(user_counts[user_counts >= min_user_ratings].index)
#     & ratings["BGGId"].isin(game_counts[game_counts >= min_game_ratings].index)
# ]

In [10]:
# === 5. Amostragem controlada (opcional, caso ainda esteja muito grande) ===
# Exemplo: seleciona 10.000 usuários aleatoriamente
# sampled_users = ratings["Username"].drop_duplicates().sample(10000, random_state=42)
# ratings = ratings[ratings["Username"].isin(sampled_users)]

In [11]:
# === 6. Criação da matriz R (usuário x jogo) ===
# Cria índices numéricos para usuários e jogos
user_index = {u: i for i, u in enumerate(ratings["Username"].unique())}
game_index = {g: i for i, g in enumerate(ratings["BGGId"].unique())}

# Mapeia IDs para índices
ratings["user_idx"] = ratings["Username"].map(user_index)
ratings["game_idx"] = ratings["BGGId"].map(game_index)

# Cria matriz esparsa
R_sparse = csr_matrix(
    (ratings["Rating"], (ratings["user_idx"], ratings["game_idx"])),
    shape=(len(user_index), len(game_index))
)

print(f"Matriz R criada: {R_sparse.shape[0]} usuários x {R_sparse.shape[1]} jogos")
print(f"Densidade: {R_sparse.nnz / (R_sparse.shape[0] * R_sparse.shape[1]):.6f}")

Matriz R criada: 217775 usuários x 19070 jogos
Densidade: 0.004077


# Filtragem Colaborativa Baseada em Modelo

### 🎯 Objetivo da seção prática

Você quer:

1. Criar um **sistema de recomendação funcional** (baseado no teu dataset BGG).
2. Aplicar uma metodologia de **validação experimental** (cross-validation, RMSE, etc).
3. Mostrar resultados interpretáveis (top-N recomendações, desempenho quantitativo).

### 🧠 1. Escolha do tipo de SR

Como você já decidiu usar as tabelas `USER_RATINGS`, `GAMES` e `MECHANICS`, o **modelo base mais apropriado** é um **Sistema de Filtragem Colaborativa Baseado em Modelos**, usando **fatoração de matrizes (Matrix Factorization)**.

👉 Esse método é o mesmo usado no *Netflix Prize* e é muito sólido para bases grandes como a tua.

---

### ⚙️ 2. Preparação dos dados

Antes de treinar o modelo, você precisa:

* Criar a **matriz ( R_{u,i} )** de avaliações dos usuários (`USER_RATINGS`);
* Tratar **ratings nulos (itens não avaliados)** como “desconhecidos” (não zero!);
* Garantir **balanceamento** (usuários e jogos com número mínimo de avaliações);
* Dividir os dados em **treino** e **teste**.

### 🧮 3. Modelo de Fatoração de Matrizes (MF)

A ideia é decompor $( R \approx P Q^T )$,
onde:

* $( P \in \mathbb{R}^{|U| \times k} )$: representa usuários em um espaço latente,
* $( Q \in \mathbb{R}^{|I| \times k} )$: representa itens (jogos),
* $( k )$: número de fatores latentes.

A predição é:
$$
\hat{r}_{ui} = p_u^T q_i
$$

O treinamento consiste em minimizar o erro quadrático regularizado:
$$
\min_{P,Q} \sum_{(u,i) \in \text{Train}} (r_{ui} - p_u^T q_i)^2 + \lambda (||p_u||^2 + ||q_i||^2)
$$


### 📈 5. Validação e análise

Você pode apresentar os resultados de:

* **RMSE** e **MAE** (precisão preditiva);
* Exemplo de **recomendações geradas** (lista top-10 para um usuário);
* Discussão sobre **vantagens e limitações práticas** do modelo (ex: escalabilidade, dados esparsos).

In [ ]:
from sklearn.model_selection import train_test_split

# Divisão treino/teste
#train, test = train_test_split(ratings, test_size=0.2, random_state=42)

#print(f"Tamanho do treino: {len(train)}, teste: {len(test)}")

Tamanho do treino: 13567030, teste: 3391758


In [16]:
from surprise import Dataset, Reader, SVD
from surprise.model_selection import cross_validate

# Preparar dados para a biblioteca
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(ratings[['Username', 'BGGId', 'Rating']], reader)

# REALIZAR UMA AMOSTRAGEM REPRESENTATIVA DA BASE PARA TREINAR O MODELO
# Selecionar os 5.000 usuários mais ativos
top_users = ratings['Username'].value_counts().head(5000).index
ratings = ratings[ratings['Username'].isin(top_users)]

# Selecionar os 2.000 jogos mais avaliados
top_games = ratings['BGGId'].value_counts().head(2000).index
ratings = ratings[ratings['BGGId'].isin(top_games)]

train, test = train_test_split(ratings, test_size=0.2, random_state=42)

# Modelo SVD (Singular Value Decomposition)
print("Treinando o modelo SVD...")
model = SVD(n_factors=50, reg_all=0.02, lr_all=0.005, random_state=42)

# Validação cruzada
print("Validando o modelo com 3-fold CV...")
results = cross_validate(model, data, measures=['RMSE', 'MAE'], cv=3, verbose=True)

print("RMSE médio:", results['test_rmse'].mean())
print("MAE médio:", results['test_mae'].mean())


Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0607  1.0603  1.0590  1.0585  1.0596  1.0596  0.0008  
MAE (testset)     0.7906  0.7906  0.7891  0.7893  0.7907  0.7900  0.0007  
Fit time          20.38   20.54   20.41   28.83   22.44   22.52   3.25    
Test time         4.76    4.26    3.76    6.23    3.82    4.57    0.91    
RMSE médio: 1.0596303313574884
MAE médio: 0.7900495903952492


In [ ]:
# === 7. Junção com tabela de mecânicas ===
# games_mech = games_modern.merge(mechanics, on="BGGId", how="left")

# # Mantém apenas os jogos que estão na matriz R
# games_mech = games_mech[games_mech["BGGId"].isin(game_index.keys())]

# # Cria DataFrame de atributos (para modelos baseados em conteúdo)
# X = games_mech.set_index("BGGId").drop(columns=["Name", "Description"], errors="ignore")

# print(f"Shape dos atributos (MECÂNICAS + METADADOS): {X.shape}")
